In [16]:
import lammps
import pymc3 as pm
import arviz as az
import numpy as np
from copy import deepcopy

from poscar_to_lammps import poscar_to_lammps
from pymatgen.io.vasp import Poscar

import sys
from contextlib import contextmanager

In [17]:
def type_shell(core_shell):
    shell_nums = []
    x = 0
    for i in core_shell.values():
        x +=1
        if i:
            x+=1
            shell_nums.append(x)
    return shell_nums

In [18]:
@contextmanager
def custom_redirection(fileobj):
    old = sys.stdout
    sys.stdout = fileobj
    try:
        yield fileobj
    finally:
        sys.stdout = old

core_shell = { 'Li': False , 'Ni': False, 'O': True}
shells = type_shell(core_shell)

# 2. Set charges: values either single number (non-core-shell)
# or a sub-dictionary containing { 'core': core_charge, 'shell': shell_charge }
charges = {'Li': +1.0,
           'Ni': +3.0,
           'O': {'core':  +0.960,
                 'shell': -2.960}}

# 3. Load a POSCAR using pymatgen and write lammps file using poscar_to_lammps
file = 'multicoords.lmp'
with open(file, 'w') as struct:
    with custom_redirection(struct):
        poscar = Poscar.from_file('../poscars/POSCAR1')
        poscar_to_lammps(poscar, core_shell, charges )

In [19]:
elements = ['Li', 'Ni', 'Oc', 'O']
pairs = [('Li','O'), ('Ni','O'), ('O','O')]
springs = ['O']

bpp_def = {'Li_O_A'   : 632.1018,
           'Li_O_rho' : 0.2906,
           'Li_O_C'   : 0.0,
           'Ni_O_A'   : 1582.000,
           'Ni_O_rho' : 0.2882,
           'Ni_O_C'   : 0.000,
           'O_O_A'    : 22844.0000,
           'O_O_rho'  : 0.0870,
           'O_O_C'    : 20.7,
           'O_k'      : 65.0,
           'O_r'      : 0.0}

bpp = deepcopy(bpp_def)

In [20]:
def init():
    lmp = lammps.Lammps(units='metal', style = 'full', args=['-log', 'none', '-screen', 'none'])
    lmp.command('read_data multicoords.lmp')
    
    lmp.command('group cores type 1 2 3')
    lmp.command('group shells type 4')
    
    lmp.command('neighbor 2.0 bin')
    lmp.command('comm_modify vel yes')

    lmp.command('kspace_style ewald 1e-6')
    lmp.command('pair_style buck/coul/long/cs 10.0') #ADD THE CS IF USING CORESHELLS NOT IF RIGID ION
    lmp.command('pair_coeff * * 0 1 0')
    
    lmp.command('bond_style harmonic')
    

    return lmp

In [21]:
def set_potentials(lmp):
    for pair in pairs:
        lmp.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]),
                                                            dex(pair[1]),
                                                            bpp['{}_{}_A'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_C'.format(pair[0], pair[1])]))
    for i, spring in enumerate(springs):
        lmp.command('bond_coeff {} {} {}'.format(i+1,
                                                 bpp['{}_k'.format(spring)],
                                                 bpp['{}_r'.format(spring)]))
    

def dex(elin): return elements.index(elin) + 1

In [38]:
lmp = init()
core_mask = [ atype not in shells for atype in lmp.system.types ]
# out = np.zeros([sum(core_mask),3])
out = np.zeros([len(index), 3])

set_potentials(lmp)

lmp.command('fix 1 cores setforce 0.0 0.0 0.0')
lmp.command('min_style cg')
lmp.command('minimize 1e-25 1e-25 5000 10000')
lmp.command('unfix 1')
lmp.run(0)
# out = lmp.system.forces[core_mask]
out = np.array([value for i, value in enumerate(lmp.system.forces) if i in index])

In [23]:
maxElement = np.amax(out)
maxElement
# np.argmax(out) #index value

1.826754674579115

In [26]:
core_mask = [atype not in shells for atype in lmp.system.types]

288

In [30]:
out

array([[ 2.66481027e-05, -2.42280174e-05, -1.26109034e-04],
       [ 2.50717486e-05, -2.07498266e-05, -1.25501671e-04],
       [ 2.64966512e-05, -2.17959285e-05, -1.28392946e-04],
       [ 2.64693547e-05, -2.22511796e-05, -1.27071905e-04],
       [ 2.63586163e-05, -2.40825609e-05, -1.23039705e-04],
       [ 2.44256313e-05, -2.03400568e-05, -1.22804291e-04],
       [ 2.58227536e-05, -2.19998603e-05, -1.24823693e-04],
       [ 2.60775067e-05, -2.21630265e-05, -1.24266395e-04],
       [ 2.26855916e-05, -2.28141783e-05, -1.23842794e-04],
       [ 2.18991105e-05, -1.87461084e-05, -1.23413988e-04],
       [ 2.31830957e-05, -2.01219273e-05, -1.25897964e-04],
       [ 2.28249224e-05, -2.04538619e-05, -1.24765175e-04],
       [ 1.98798704e-05, -2.23072475e-05, -1.21485688e-04],
       [ 1.82100716e-05, -1.89352161e-05, -1.21536640e-04],
       [ 1.98754533e-05, -2.04706112e-05, -1.23728435e-04],
       [ 1.97477092e-05, -2.06051563e-05, -1.22636794e-04],
       [-3.35860396e-07,  5.62087110e-07

In [13]:
core_mask = [atype not in shells for atype in lmp.system.types]
out = np.zeros([sum(core_mask),3])
# out = lmp.system.forces[core_mask]
expected = np.zeros([sum(core_mask),3])

In [14]:
expected.shape

(192, 3)

In [33]:
index = [ i for i, atype in enumerate(lmp.system.types) if atype not in shells]